## Elastic Net Model Accuracy Comparison for some Stocks

---

|  |  |
|:---|:---|
|**Reading Time** |  10 minutes |
|**Packages used** | Defined with relevant DocStrings within repository   |
|**Model used** | Model already trained stored in the directory ./model/output|
|**Outcome** | Comparison of the model accuracy on the real dataset | 

---

In [1]:
import os
import pickle

import pandas as pd
import numpy as np
import yfinance as yf

import warnings
warnings.filterwarnings("ignore") # just to remove clutter in the notebook

from datetime import date, datetime, timedelta
from sklearn.linear_model import ElasticNet

In [2]:
os.chdir("..")

In [3]:
from model.utils.pre_processing import create_features_and_target_split, standardize_and_limit_outliers_returns_first_try, pre_process_for_test_set

In [4]:
ticker = "SBIN.NS"
end_date = datetime.today()
start_date = date(2016, 1, 1)
df_hist = yf.download(ticker, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [6]:
param = {'training_end': end_date - timedelta(seconds=2 * 365.2425 * 24 * 60 * 60),
             'validation_end': end_date - timedelta(seconds=1 * 365.2425 * 24 * 60 * 60),
             'past_day_returns_for_predicting': 10}

In [7]:
df_hist['daily_returns'] = df_hist['Adj Close'].pct_change()
df_hist = df_hist.dropna()
    # create rolling dataset
dt_model = create_features_and_target_split(pd.DataFrame(df_hist['daily_returns']),
                                                param['past_day_returns_for_predicting'])

In [9]:
X_train, Y_train, X_val, Y_val = standardize_and_limit_outliers_returns_first_try(dt_model, 10, **param)

In [10]:
model = ElasticNet(alpha=0, l1_ratio=0)
model.fit(X_train, Y_train)

ElasticNet(alpha=0, l1_ratio=0)

In [9]:
df['actual_returns'] = df['Adj Close'].pct_change()
df = df.dropna()
df['predicted_returns'] = ""

NameError: name 'df' is not defined

In [10]:
df_training = df[df.index <= param['training_end']]

NameError: name 'df' is not defined

In [23]:
rol_freq_for_prediction = 60
df['rolling_returns'] = df['actual_returns']
for i in range(rol_freq_for_prediction, len(df_training)):
    features_ls = np.array(df_training.iloc[i - rol_freq_for_prediction:i, 0:df_training.shape[1]]['actual_returns'].to_list())
    features_ls = features_ls.reshape(-1,1)
    new_scaled_ls = scaler.transform(features_ls.T)
    predicted_return = model.predict(new_scaled_ls)[0]
    df.iloc[i, df.columns.get_loc('rolling_returns')] = predicted_return
    df.iloc[i, df.columns.get_loc('predicted_returns')] = predicted_return

In [15]:
rol_freq_for_prediction = 60
for i in range(rol_freq_for_prediction, len(df_training)):
    features_ls = np.array(df_training.iloc[i - rol_freq_for_prediction:i, 0:df_training.shape[1]]['actual_returns'].to_list())
    features_ls = features_ls.reshape(-1,1)
    new_scaled_ls = scaler.transform(features_ls.T)
    predicted_return = model.predict(new_scaled_ls)[0]
    df_training.iloc[i, df_training.columns.get_loc('predicted_returns')] = predicted_return

NameError: name 'scaler' is not defined

In [16]:
df_training.to_clipboard()

In [11]:
Y_train.shape

(1243,)

In [11]:
y_pred = model.predict(X_train)

In [18]:
df = pd.DataFrame(y_pred)

In [22]:
df.to_clipboard()

In [20]:
df_training.to_clipboard()

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [13]:
mse = mean_squared_error(Y_train, y_pred)

In [13]:
r2 = r2_score(Y_train, y_pred)

In [14]:
r2

0.0176554093511212

In [15]:
df_train = pd.DataFrame(Y_train)

In [18]:
df_pred = pd.DataFrame(y_pred)

In [17]:
df_train.to_clipboard()

In [19]:
df_pred.to_clipboard()

In [25]:
xt = pd.DataFrame(X_train)

In [26]:
xt.to_clipboard()